In [1]:
import sys
sys.path.insert(0, "../")
sys.path.insert(0, "../signease-master/")
import nsp
import numpy as np
import torch
import utils
import utils.optm as optm
import utils.lossfunc as lf
from importlib import reload
from circuitOptimizer import *
from identicalOnsiteCircuit import *
from modelHamiltonian import *


In [2]:
# # Set parameters of the problem 
locality = 2
localDim = 4
X = np.random.randn(16,16)
X = (X + X.T)/2

In [3]:
H = Hamiltonian(locality, localDim)
H.localTerm = np.random.randn(4,4)
myMeasure = Measure(localDim=localDim, measureType={'name':"fro"})
optimizer = Optimizer(myMeasure, H)
initCircuit = IdenticalOnsiteCircuit(localDim=localDim)
initCircuit.initRandom()
optimizer.initCircuit = initCircuit


In [27]:
froCircuit, froReport = optimizer.optimize()


-----
Started conjugate gradient optimisation:
-----
obj val | grad norm | step size
0.947277424070621 0.0193297257391442 0.047451658820285895
11.369305749115622 38.18353875991042 0.0
0.9487270156512477 0.010462772332656065 0.11862837198525188
1.143579826875314 0.5488781498108628 0.0
0.9473035638978273 0.06407869158731251 0.21202247792560108
8.107822635153022 324.5877238815555 0.0
0.9473245047814975 0.06442472888871324 0.03487496842707269
7.491288956878652 304.2577284993559 0.0
0.9500085111787944 0.006239318187456211 0.03301950041912103
0.9464326367372244 0.03980071882860651 58.55207480203687
0.9464288769297742 0.03826163032454555 0.003411901750747858
17.499691554226295 456.1864475885521 0.0
0.9468319089398447 0.05508885339060245 0.04167507155227841
1.673368802413492 6.513723328372071 0.0
0.9467791117416327 0.025385701209037114 0.102577939885903
0.9464289132909189 0.03815782981810548 0.6316919729031956
4.2467353057160295 29.70618316810267 0.0
0.9518707731079514 0.002662490960414038 0.

In [19]:
# myMeasure.measureType = {'name':'ell1','epsilon':0}
# optimizer.initCircuit = initCircuit

# l1Circuit, l1Report = optimizer.optimize(max_steps = 500)
